In [39]:
font_list = [ 
    '/usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans-Bold.ttf',
    '/usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSansMono-Bold.ttf',
    '/usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSansMono.ttf',
    '/usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf',
    '/usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSerif-Italic.ttf',
    '/usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSerif.ttf',
 ]
 
import random
import PIL
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

font = ImageFont.truetype(random.choice(font_list), size=random.randint(100, 200))
font

In [40]:
import os

if not os.path.exists("./data/test"): os.mkdir("./data/test")
if not os.path.exists("./data/train"): os.mkdir("./data/train")

Пилим пикчу на составные и работаем с ними как с отдельными изображениями

In [41]:
# PIL.Image.MAX_IMAGE_PIXELS = 900000000

# img = Image.open('/home/owner/Documents/DEV/DL/hw-letters-xCthulhuFtagn/letters_58.png')

# for i in range(100):
#     for j in range(100):
#         new_img = img.crop((300*i,300*j, 300*(i+1), 300*(j+1)))
#         new_img.save(f"./data/test/letter_{(i+1) + j*100}.jpeg")

Генерим свои данные

In [42]:
# # https://stackoverflow.com/questions/75344148/trying-to-use-python-pil-imagedraw-textbbox-because-of-textsize-deprecation-but
# from string import ascii_uppercase
# from random import randint
# import math
# import numpy as np

# classes = (set(ascii_uppercase) - {'N', 'Z'} - {'M', 'W'}).union({"N_Z", "M_W"})
# for letter in classes:
#     for i in range(100):
#         empty = Image.new('RGBA', (300, 300), (255, 255,255, 255))
#         overlay = Image.new('RGBA', (300, 300), (255, 255, 255, 0))
        
#         empty_draw = ImageDraw.Draw(empty)
#         n_circles = randint(10, 40)
#         for _ in range(n_circles):
#             circle = Image.new('RGBA', (300, 300), (255, 255,255, 0))
#             circle_draw = ImageDraw.Draw(circle)
#             color = (random.randint(0, 200), 
#                      random.randint(0, 200), 
#                      random.randint(0, 200),
#                      random.randint(50, 70))
#             x = randint(10, 290)
#             y = randint(10, 290)
#             radius = randint(1, 35)
#             xy = [x - radius, y - radius, x + radius, y + radius]
#             circle_draw.ellipse(xy, fill=color)
#             empty = Image.alpha_composite(empty, circle)
        
#         overlay_draw = ImageDraw.Draw(overlay)
#         spin_angle = randint(0, 359)
#         radians = math.radians(spin_angle)
#         spread = 0
#         x, y = 150 + randint(-spread, spread), 150 + randint(-spread, spread)
        
#         if letter == "N_Z": label = "N" if np.random.random() < 0.5 else "Z"
#         elif letter == "M_W": label = "M" if np.random.random() < 0.5 else "W"
#         else: label = letter
        
#         overlay_draw.text(
#             (x, y),
#             text=label,
#             font=font,
#             fill=(randint(0, 255), randint(0, 255), randint(0, 255), 255),
#         )
        
#         new_img = Image.alpha_composite(empty, overlay.rotate(spin_angle))
#         new_img = new_img.convert('RGB')
#         new_img.save(f"./data/train/{letter}_{i}.png", 'PNG')


Херачим модель

In [43]:
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import os
import numpy as np
from string import ascii_uppercase

class DatasetLetters(Dataset):
    def __init__(self, root='./data', train=True, transform=None):
        self.root = root
        self.transform = transform
        self.classes = (set(ascii_uppercase) - {'N', 'Z'} - {'M', 'W'}).union({"N_Z", "M_W"})
        self.classes = dict(zip(range(len(self.classes)), self.classes))
        self.len = len(os.listdir(self.root + "/train"))

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        # load image
        # transform image
        # return dict with image and class label
        letter = self.classes[index // 100]
        sub_index = index % 100
        file_name = f"{letter}_{sub_index}.png"
        pic = cv2.imread(self.root + "/train/" + file_name)
        # labels = torch.zeros(24).type(torch.LongTensor)
        # labels[index // 100] = 1
        return {"image": self.transform(pic), "label": torch.Tensor([index // 100]).type(torch.LongTensor)}

In [44]:
import torch.nn as nn
import torch.nn.functional as F

class LetterCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=3) #100x100x6
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=1, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=4, stride=4) #25x25x4
        self.lin = nn.Linear(in_features=625, out_features=100)
        self.final = nn.Linear(in_features=100, out_features=24)
    
    def forward(self, x : torch.Tensor):
        # print(f"x: {x.shape}")
        x.to("cuda:0")
        x = F.leaky_relu(self.conv1(x))
        x = self.pool1(x)
        x = F.leaky_relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.lin(x.flatten(2)))
        return self.final(x)

In [45]:
from torchsummary import summary

summary(model=LetterCNN().to("cuda:0"), input_size=(3,300,300), batch_size=128)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [128, 6, 300, 300]             168
         MaxPool2d-2         [128, 6, 100, 100]               0
            Conv2d-3         [128, 1, 100, 100]              55
         MaxPool2d-4           [128, 1, 25, 25]               0
            Linear-5              [128, 1, 100]          62,600
            Linear-6               [128, 1, 24]           2,424
Total params: 65,247
Trainable params: 65,247
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 131.84
Forward/backward pass size (MB): 596.43
Params size (MB): 0.25
Estimated Total Size (MB): 728.52
----------------------------------------------------------------


In [46]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input.shape, target.shape)
output = loss(input, target)
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()

torch.Size([3, 5]) torch.Size([3])


In [47]:
# from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
import os
import pandas as pd

class Trainer:
    def __init__(self, model, optimizer, train_dataset, batch_size=128):
        self.model = model
        self.optimizer = optimizer
        self.train_dataset = train_dataset
        self.lossF = nn.CrossEntropyLoss()

        self.batch_size = batch_size

        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = self.model.to(self.device)

        self.global_step = 0

    def save_checkpoint(self, path):
        torch.save(self.model.state_dict(), path)
        
    def load_checkpoint(self, path):
        self.model = torch.load(path)

    def train(self, num_epochs):
        model = self.model
        optimizer = self.optimizer

        train_loader = DataLoader(self.train_dataset, shuffle=True, pin_memory=True, batch_size=self.batch_size)

        for epoch in range(num_epochs):
            model.train()
            for batch in tqdm(train_loader):
                x = batch["image"].to(self.device)
                y = torch.Tensor(batch["label"]).to(self.device)
                
                logits = model.forward(x)
                print(logits.shape, y.shape)
                
                loss = self.lossF(logits, y)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                self.global_step += 1
                
    def classify(self):
        path = self.path + "/test"
        
        classes = self.train_dataset.classes
        
        letters_df = pd.read_csv("/home/owner/Documents/DEV/DL/hw-letters-xCthulhuFtagn/letters.csv")
        
        letters_df.values = 0
        
        for filename in os.listdir(path):
            
            pic = cv2.imread(filename)
            logits = self.model(pic)
            
            letter = classes[np.argmax(logits)]
            if letter == "N/Z":
                if np.random.random() < 0.5: letters_df["N"] += 1
                else: letters_df["Z"] += 1
            elif letter == "M/W":
                if np.random.random() < 0.5: letters_df["M"] += 1
                else: letters_df["W"] += 1


In [48]:
from torchvision import transforms

transform_to_tensor = transforms.Compose([
    transforms.ToTensor()
])
train_dataset = DatasetLetters(transform=transform_to_tensor)
model = LetterCNN()

trainer = Trainer(model, torch.optim.Adam(model.parameters()), train_dataset)

In [49]:
trainer.train(10000)

  0%|          | 0/19 [00:00<?, ?it/s]

torch.Size([128, 1, 24]) torch.Size([128, 1])


RuntimeError: Expected target size [128, 24], got [128, 1]

In [53]:
import torch.nn as nn
import torch
loss = nn.CrossEntropyLoss()

inputs = torch.rand(128, 3, 32)
targets = torch.ones(128, 32)
print(inputs.shape, targets.shape)
loss(inputs, targets.long())

torch.Size([128, 3, 32]) torch.Size([128, 32])


tensor(1.1245)